In [38]:
#YOLOv5 레퍼지토리 클론
!git clone https://github.com/ultralytics/yolov5

In [39]:
#YOLOv5 라이브러리 설치
!cd yolov5; pip install -r requirements.txt

In [2]:
# 필요한 라이브러리 불러오기
import os
import json
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [4]:
#이미지 모두 yolo 형식에 맞는 jpg로 변환
%cd C:/Users/mjkwo/yolov5/data/images
!ren *.* *.jpg*

C:\Users\mjkwo\yolov5\data\images


In [5]:
# labelme를 통해 추출한 JSON 파일을 YOLO 형식

# JSON 파일이 있는 디렉토리 경로
json_dir = r'C:\Users\mjkwo\yolov5\data\labels'

# 변환된 YOLO 형식의 텍스트 파일을 저장할 디렉토리 경로
output_dir = r'C:\Users\mjkwo\yolov5\data\yolo_labels'

# 만약 저장 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 디렉토리 내의 모든 JSON 파일 처리
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        # JSON 파일 읽기
        json_path = os.path.join(json_dir, filename)
        with open(json_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        # YOLO 형식의 텍스트 파일 경로 생성
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        txt_path = os.path.join(output_dir, txt_filename)

        # 이미지의 폭과 높이 추출
        image_width = data['imageWidth']
        image_height = data['imageHeight']

        # YOLO 형식으로 변환하고 텍스트 파일에 저장
        with open(txt_path, 'w') as yolo_file:
            for shape in data['shapes']:
                label = 0  # 클래스 식별자를 0으로 설정 (Defect 대신)
                points = shape['points']

                # YOLO 형식으로 좌표 변환 (0.0 ~ 1.0)
                x_center = (points[0][0] + points[1][0]) / (2 * image_width)
                y_center = (points[0][1] + points[1][1]) / (2 * image_height)
                width = abs(points[1][0] - points[0][0]) / image_width
                height = abs(points[1][1] - points[0][1]) / image_height

                # YOLO 형식으로 텍스트 작성
                yolo_line = f"{label} {x_center} {y_center} {width} {height}\n"
                yolo_file.write(yolo_line)

print("모든 JSON 파일을 YOLO 형식의 텍스트 파일로 변환하였습니다.")

모든 JSON 파일을 YOLO 형식의 텍스트 파일로 변환하였습니다.


In [7]:
#폴더 안 이미지들 모두 리스트로 읽어옴

%cd /
from glob import glob

img_list = glob('C:/Users/mjkwo/yolov5/data/images/*.jpg')

print(len(img_list))

C:\
275


In [8]:
#train, valid 8:2로 나눔
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

220 55


In [9]:
#train valid text 파일로 만들어줌
with open('C:/Users/mjkwo/yolov5/data/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
    
with open('C:/Users/mjkwo/yolov5/data/val.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

In [10]:
#앞서 만들어준 data.yaml 파일의 train, val 경로를 재설정해준다
import yaml

with open('C:/Users/mjkwo/yolov5/data/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
print(data)

data['train'] = 'C:/Users/mjkwo/yolov5/data/train.txt'
data['val'] = 'C:/Users/mjkwo/yolov5/data/val.txt'

with open('C:/Users/mjkwo/yolov5/data/data.yaml', 'w') as f:
    yaml.dump(data, f)
    
print(data)

{'names': ['Defect'], 'nc': 1, 'train': 'C:/Users/mjkwo/yolov5/data/train.txt', 'val': 'C:/Users/mjkwo/yolov5/data/val.txt'}
{'names': ['Defect'], 'nc': 1, 'train': 'C:/Users/mjkwo/yolov5/data/train.txt', 'val': 'C:/Users/mjkwo/yolov5/data/val.txt'}


In [1]:
#모델 학습 
%cd C:/Users/mjkwo/yolov5/
    
!python train.py --img 416 --batch 16 --epochs 50 --data C:/Users/mjkwo/yolov5/data/data.yaml --cfg C:/Users/mjkwo/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name gun_yolov5s_results
print("끝났습니다")

C:\Users\mjkwo\yolov5
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
끝났습니다


train: weights=yolov5s.pt, cfg=C:/Users/mjkwo/yolov5/models/yolov5s.yaml, data=C:/Users/mjkwo/yolov5/data/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=gun_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  v7.0-215-ga6659d0 Python-3.11.4 torch-2.0.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.

In [35]:
#1호기 오토라벨링 
%cd C:/Users/mjkwo/yolov5/

!python detect.py --source C:/Users/mjkwo/yolov5/data/unit_1 --weights C:/Users/mjkwo/yolov5/runs/train/gun_yolov5s_results8/weights/best.pt --save-txt --exist-ok --project C:/Users/mjkwo/yolov5/data/unit_1_label

C:\Users\mjkwo\yolov5


detect: weights=['C:/Users/mjkwo/yolov5/runs/train/gun_yolov5s_results8/weights/best.pt'], source=C:/Users/mjkwo/yolov5/data/unit_1, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=C:/Users/mjkwo/yolov5/data/unit_1_label, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-215-ga6659d0 Python-3.11.4 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/943 C:\Users\mjkwo\yolov5\data\unit_1\002_20200622_203053(2).jpg: 608x640 3 Defects, 115.2ms
image 2/943 C:\Users\mjkwo\yolov5\data\unit_1\002_20200622_203057(3).jpg: 608x640 3 Defects, 113.2ms
image 3/943 C:\Users\mjkwo\yolov5\data\unit_1\002_20200622_203101(3).jpg: 6

In [17]:
%cd C:/Users/mjkwo/yolov5/data/unit_1
!ren *.* *.jpg*

C:\Users\mjkwo\yolov5\data\unit_1


In [36]:
%cd /
from glob import glob

img_list = glob('C:/Users/mjkwo/yolov5/data/images/*.jpg')

print(len(img_list))

C:\
943


In [37]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

754 189


In [38]:
with open('C:/Users/mjkwo/yolov5/data/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
    
with open('C:/Users/mjkwo/yolov5/data/val.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

In [39]:
import yaml

with open('C:/Users/mjkwo/yolov5/data/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
print(data)

data['train'] = 'C:/Users/mjkwo/yolov5/data/train.txt'
data['val'] = 'C:/Users/mjkwo/yolov5/data/val.txt'

with open('C:/Users/mjkwo/yolov5/data/data.yaml', 'w') as f:
    yaml.dump(data, f)
    
print(data)

{'names': ['Defect'], 'nc': 1, 'train': 'C:/Users/mjkwo/yolov5/data/train.txt', 'val': 'C:/Users/mjkwo/yolov5/data/val.txt'}
{'names': ['Defect'], 'nc': 1, 'train': 'C:/Users/mjkwo/yolov5/data/train.txt', 'val': 'C:/Users/mjkwo/yolov5/data/val.txt'}


In [40]:
#오토라벨링 성능측정
%cd C:/Users/mjkwo/yolov5/
    
!python train.py --img 416 --batch 16 --epochs 50 --data C:/Users/mjkwo/yolov5/data/data.yaml --cfg C:/Users/mjkwo/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name gun_yolov5s_unit1_results
print("끝났습니다")

C:\Users\mjkwo\yolov5
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
끝났습니다


train: weights=yolov5s.pt, cfg=C:/Users/mjkwo/yolov5/models/yolov5s.yaml, data=C:/Users/mjkwo/yolov5/data/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=gun_yolov5s_unit1_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  v7.0-215-ga6659d0 Python-3.11.4 torch-2.0.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degr

In [41]:
%cd /
from glob import glob

img_list = glob('C:/Users/mjkwo/yolov5/data/images/*.jpg')

print(len(img_list))

C:\
805


In [42]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

644 161


In [43]:
with open('C:/Users/mjkwo/yolov5/data/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
    
with open('C:/Users/mjkwo/yolov5/data/val.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

In [44]:
import yaml

with open('C:/Users/mjkwo/yolov5/data/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
print(data)

data['train'] = 'C:/Users/mjkwo/yolov5/data/train.txt'
data['val'] = 'C:/Users/mjkwo/yolov5/data/val.txt'

with open('C:/Users/mjkwo/yolov5/data/data.yaml', 'w') as f:
    yaml.dump(data, f)
    
print(data)

{'names': ['Defect'], 'nc': 1, 'train': 'C:/Users/mjkwo/yolov5/data/train.txt', 'val': 'C:/Users/mjkwo/yolov5/data/val.txt'}
{'names': ['Defect'], 'nc': 1, 'train': 'C:/Users/mjkwo/yolov5/data/train.txt', 'val': 'C:/Users/mjkwo/yolov5/data/val.txt'}
